In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')
tf.get_logger().setLevel('ERROR')

#import the data set data/hatespeech.csv
data_path = os.path.join(os.getcwd(), 'data/hatespeech.csv')



ModuleNotFoundError: No module named 'tensorflow_text'